In [45]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import pandas as pd

import gc #garbage collector
import warnings
warnings.filterwarnings('ignore')


In [46]:
import gc #garbage collector
from playsound import playsound
#!pip install playsound

import time

start_time = time.time()

In [47]:
path_data = "..\\data\\"
figures = "..\\figures\\"
netflix = "..\\data\\netflix-prize-data\\"

### Lectura del dataset User-Rating-movieid

In [48]:
%%time
df1 = pd.read_csv(netflix + 'combined_data_1_with_movie_id.csv', dtype={'Rating': np.int8, 'movie_id': np.int16})


Wall time: 10.2 s


In [49]:
print(df1.shape)
df1.head()

(24053764, 3)


,User,Rating,Movie_id
0,1488844,3,1
1,822109,5,1
2,885013,4,1
3,30878,4,1
4,823519,3,1


# df1.User.value_counts().nlargest(5) 

In [50]:
df1.User.value_counts().nlargest(5) 

305344     4467
387418     4422
2439493    4195
1664010    4019
2118461    3769
Name: User, dtype: int64

In [51]:
df1.User.value_counts().nsmallest(5)

2547574    1
1839857    1
1457200    1
2170088    1
1610759    1
Name: User, dtype: int64

# np.logical_or
```
x = np.arange(5)
np.logical_xor(x < 1, x > 3)
array([ True, False, False, False,  True])
```
alt96

```
x = np.arange(5)
np.logical_and(x>1, x<4)
array([False, False,  True,  True, False])
```

In [52]:
#vamos a descartar aquellos usuarios que califican poco o mucho
#Condicion de filtro {{¨´´´¨¨¨¨**¨*¨¨}}
mask_usuarios_descartables = np.logical_or(df1.User.value_counts() <=5, df1.User.value_counts() > 100)

# mask_usuarios_descartables[mask_usuarios_descartables].index.values

In [53]:
#Filtro aplicado y llevado a index
usuarios_descartables = mask_usuarios_descartables[mask_usuarios_descartables].index.values

## mascara_descartables = df1.User.isin(usuarios_descartables)


In [54]:
#utilizo funcion de pandas isin
mascara_descartables = df1.User.isin(usuarios_descartables)


df1 = df1[~mascara_descartables]

In [55]:
print(df1.shape)
df1 = df1[~mascara_descartables]
print(df1.shape)

(24053764, 3)
(10658448, 3)


### Movie-id-name

In [56]:
with open(netflix + "movie_titles.csv") as f:
    print(f)
    #for i in range(10):
    #    print(f.readline())


<_io.TextIOWrapper name='..\\data\\netflix-prize-data\\movie_titles.csv' mode='r' encoding='cp1252'>


In [57]:
df_title = pd.read_csv(netflix + "movie_titles.csv",encoding = "cp1252",index_col = 0, header = None, usecols = [0,2], names = ['Movie_Id', 'Name'])

In [58]:
df_title.head()

,Name
Movie_Id,
1,Dinosaur Planet
2,Isle of Man TT 2004 Review
3,Character
4,Paula Abdul's Get Up & Dance
5,The Rise and Fall of ECW


In [59]:
df1.Movie_id.value_counts().nlargest(5) 

1905    128150
2152    104851
3860    102968
4432    101720
3938     99295
Name: Movie_id, dtype: int64

# filtro con menos de 3mil usuarios por pelicula
# df1.Movie_id.value_counts() <= 3000

In [60]:
mask_items_descartables = df1.Movie_id.value_counts() <= 3000
items_descartables = mask_items_descartables[mask_items_descartables].index.values
mascara_descartables = df1.Movie_id.isin(items_descartables)
print(df1.shape)
df1 = df1[~mascara_descartables]
print(df1.shape)

(10658448, 3)
(9051531, 3)


# segunda parte

### Matriz de Utilidad

# Esto es un dataset y no una matriz de utilidad


Para poder continuar, es necesario llevarlo a una matriz de utilidad.

Sin embargo, hay que tener cuidado. Tenemos del orden de ~1000 ítems y ~500 mil usuarios, lo que da una matriz de 500.000.000 entradas. Pero la mayoría de estas entradas son 0, ya que solo contamos con aproxidamente 10.000.000 calificaciones (estos valores se modifican según cómo filtremos el dataset). Es decir, aproximadamente una de cada 50 posibles entradas. Si guardásemos esta información en un dataset común, casi seguro llenaríamos nuestra memoria RAM. Por suerte, existen los tipos de datos "ralos" (sparse), que hacen esto de manera eficiente.

Puedes leer más aquí.

In [61]:
df1.head(2)

,User,Rating,Movie_id
5098,824097,2,8
5099,2630686,5,8


In [62]:
usuarios = df1.User.unique()
items = df1.Movie_id.unique()


¿Cuantos son?

In [63]:
n_usuarios = len(usuarios)
n_items = len(items)
print(n_usuarios, n_items)

320181 502


# Creamos una matriz sparse (rala) de ceros, de `n_usuarios` filas y `n_items` columnas.

In [64]:
matriz_utilidad = pd.DataFrame(np.zeros((n_usuarios, n_items)), index =  usuarios, columns = items).astype(pd.SparseDtype("float"))
matriz_utilidad.head()

,8,17,28,30,33,58,83,97,108,111,...,4402,4418,4420,4432,4472,4479,4488,4490,4492,4496
824097,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2630686,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
644003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
229220,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2580478,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Calculo auxiliar como rellenar la matriz!

* Filtro
* values
* pelicula matirz_utilidad[28].loc[filtro.User.values]

In [65]:
filtro = df1[df1.Movie_id == 28]

In [66]:
filtro.Rating.values

array([5, 3, 4, ..., 4, 5, 5], dtype=int8)

In [67]:
filtro.Rating.values

array([5, 3, 4, ..., 4, 5, 5], dtype=int8)

In [68]:
matriz_utilidad[28].loc[filtro.User.values]

2626356    0.0
452013     0.0
1661657    0.0
1778432    0.0
2557878    0.0
          ... 
2090969    0.0
2249402    0.0
984872     0.0
528953     0.0
1595055    0.0
Name: 28, Length: 16424, dtype: Sparse[float64, nan]

In [69]:
 matriz_utilidad[28] = matriz_utilidad[28].sparse.to_dense()

In [70]:
matriz_utilidad[28].loc[filtro.User.values] = 5

In [71]:
matriz_utilidad[28].loc[filtro.User.values] 

2626356    5.0
452013     5.0
1661657    5.0
1778432    5.0
2557878    5.0
          ... 
2090969    5.0
2249402    5.0
984872     5.0
528953     5.0
1595055    5.0
Name: 28, Length: 16424, dtype: float64

In [72]:
matriz_utilidad = pd.DataFrame(np.zeros((n_usuarios, n_items)), index =  usuarios, columns = items).astype(pd.SparseDtype("float"))
matriz_utilidad.head()

,8,17,28,30,33,58,83,97,108,111,...,4402,4418,4420,4432,4472,4479,4488,4490,4492,4496
824097,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2630686,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
644003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
229220,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2580478,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [73]:
### Iteramos sobre cada columna
for column in matriz_utilidad:
    # guardo los tipos de la columna para utilizarlo despues
    spdtypes = matriz_utilidad.dtypes[column]

    # convierto la matriz sparse a densa para poder llenarla de valores
    matriz_utilidad[column] = matriz_utilidad[column].sparse.to_dense()
    
    ### recorro cada movie id y filtro por esas calificaciones
    mask_calificaciones_columna = df1.Movie_id == column
    calificaciones_columna = df1[mask_calificaciones_columna]
    
    # lleno la matriz de utilidad con las calificaciones por pelicula por usuario
    matriz_utilidad[column].loc[calificaciones_columna.User.values] = calificaciones_columna.Rating.values

    # vuelvo a la matriz de utilidad
    matriz_utilidad[column] = matriz_utilidad[column].astype(spdtypes)

In [74]:
matriz_utilidad

,8,17,28,30,33,58,83,97,108,111,...,4402,4418,4420,4432,4472,4479,4488,4490,4492,4496
824097,2.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,0.0
2630686,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
644003,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
229220,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
2580478,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708048,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
175653,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
1602517,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2478092,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0


In [75]:
matriz_utilidad.columns = df_title.loc[matriz_utilidad.columns]['Name'].values
matriz_utilidad.head()

,What the #$*! Do We Know!?,7 Seconds,Lilo and Stitch,Something's Gotta Give,Aqua Teen Hunger Force: Vol. 1,Dragonheart,Silkwood,Mostly Martha,Spartan,Duplex (Widescreen),...,A Nightmare on Elm Street,Titan A.E.,Against the Ropes,The Italian Job,Love Actually,Ordinary People,Wonder Boys,Ned Kelly,Club Dread,Farewell My Concubine
824097,2.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,0.0
2630686,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
644003,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
229220,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
2580478,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#https://stackoverflow.com/questions/60956268/save-sparse-pandas-dataframe-to-different-file-types

In [80]:
df1.to_pickle(netflix+"df_filtrado.pkl.zip")


In [77]:
matriz_utilidad.to_pickle(netflix+"matriz_utilidad.pkl.zip")


In [ ]:
from scipy import sparse
import scipy.sparse

data_sparse = sparse.csr_matrix(matriz_utilidad)

### Normalizacion

Ésta es la idea de normalizar los vectores de usuario para que un usuario con muchas calificaciones contribuya menos a cualquier calificación individual. Esto quiere decir que un me gusta de un usuario al que solo le han gustado 10 artículos es más valioso para nosotros que un me gusta de alguien a quien le gusta todo lo que encuentra.

,User,Rating,Movie_id
5098,824097,2,8
5099,2630686,5,8
5100,644003,3,8
5103,229220,4,8
5105,2580478,4,8


In [ ]:
from scipy import sparse
data_sparse = sparse.csr_matrix(matriz_utilidad) ##convierto a tipo de matriz sparse csr parte de library sciyp
#https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html

magnitude = np.sqrt(data_sparse.power(2).sum(axis=1)) #la maginitud son los ratings sumados al cuadrado
matriz_utilidad = matriz_utilidad.divide(magnitude, axis='index')
matriz_utilidad

# sklearn.metrics.pairwise import cosine_similarity
# sparse.crs_matrix
# data, sparse transpose

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def calculate_similarity(data_items):
    """Calculo la similaridad coseno para el par fila-columna de una matriz sparse. 
    Retorna nuevo DF con similaridades.
    """
    
    data_sparse = sparse.csr_matrix(data_items) # Esto solo convierte el dataframe en una matriz rala de scipy. Compress Sparse rows matrix
    similarities = cosine_similarity(data_sparse.transpose()) #para devolver similaridad columna columna
    sim = pd.DataFrame(data=similarities, index= data_items.columns, columns= data_items.columns)
    return sim

```
pairwise dense output:
[[ 1.          0.40824829  0.40824829]
[ 0.40824829  1.          0.33333333]
[ 0.40824829  0.33333333  1.        ]]

pairwise sparse output:
(0, 1)  0.408248290464
(0, 2)  0.408248290464
(0, 0)  1.0
(1, 0)  0.408248290464
(1, 2)  0.333333333333
(1, 1)  1.0
(2, 1)  0.333333333333
(2, 0)  0.408248290464
(2, 2)  1.0
If you want column-wise cosine similarities simply transpose your input matrix beforehand:

A_sparse.transpose()
```

In [ ]:
# Build the similarity matrix
data_matrix = calculate_similarity(matriz_utilidad)
data_matrix

### Peliculas parecidas

In [ ]:
print(data_matrix.loc["Being John Malkovich"].nlargest(25))

Vecinos. similares

### construir matriz de vecinos


In [ ]:
data_matrix.loc[:,"Dogma"].sort_values(ascending=False)

# matriz vecinos
```
n_neighbours = 5
data_neighbours = pd.DataFrame(index=data_matrix.columns, columns=range(n_neighbours))
for column in data_matrix.columns:
    data_neighbours.loc[column,:] = data_matrix.loc[:,column].sort_values(ascending=False)[:n_neighbours].index
data_neighbours
```

In [ ]:
n_neighbours = 5
data_neighbours = pd.DataFrame(index=data_matrix.columns, columns=range(n_neighbours))
for column in data_matrix.columns:
    data_neighbours.loc[column,:] = data_matrix.loc[:,column].sort_values(ascending=False)[:n_neighbours].index
data_neighbours

## A partir de Users

In [ ]:
user = np.random.choice(list(matriz_utilidad.index))
# user = 1329595  # Descomentar este usuario y ver sus resultados.
# user = 1208602  # Descomentar este usuario y ver sus resultados.
print(user)
df_user = df1[df1.User == user]
df_user['Title'] = df_title.loc[df1[df1.User == user].Movie_id]['Name'].values
df_user

### 1. User_likes

Me quedo con las películas que le puso más de 4.

Estas son las películas que vio. Tener en cuenta que no hay que recomendarle algo que ya vio

In [ ]:

known_user_likes = df_user['Title'].values
known_user_likes = df_user[df_user.Rating > 4]['Title'].values # puedes resolver aqui el ejercicio.
known_user_likes

### 2. Matriz vecinos con usuarios
Utilizo la matriz de vecinos sólo con las películas que vio mi usuario

In [ ]:
most_similar_to_likes = data_neighbours.loc[known_user_likes]
most_similar_to_likes

### 3. Filtro más similares
Las convierto en listaa

In [ ]:
similar_list = most_similar_to_likes.values.tolist()
similar_list = list(set([item for sublist in similar_list for item in sublist]))
similar_list

#### Matriz de utilidad de las calificaciones que vio
matriz de utilidad las calificaciones de las películas que vio (que están normalizados) y las de las películas que no vio. Como no las vio, son cero.

In [ ]:
user_vector = matriz_utilidad.loc[user].loc[similar_list]
user_vector

#### Vecinos con lista filtrada

Filtro la matriz original por similar list y tengo entonces esas peliculas

In [ ]:
neighbourhood = data_matrix[similar_list].loc[similar_list]
neighbourhood

#### Producto escalar
es el producto escalar de la matriz "vecindario" con las calificaciones. 
devuelvo scores de similaridad

In [ ]:
score = neighbourhood.dot(user_vector).div(neighbourhood.sum(axis=1))
score

In [ ]:
score = score.drop(known_user_likes)
score

In [ ]:
print(known_user_likes)
print(score.nlargest(20))

In [76]:
playsound(figures+"war.mp3")
gc.collect()

end = time.time()
hours, rem = divmod(end-start_time, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

00:01:18.08
